In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.v2 as transforms_v2  # CutMix를 위한 v2 transforms 추가
import sys
import os
import torch
import time
import random
import numpy as np
import wandb
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
from tools.tool import EarlyStopping
from models.resnet import resnet18, resnet34, resnet50

wandb.login(key="ef091b9abcea3186341ddf8995d62bde62d7469e")
wandb.init(project="PBL-2", name="resnet50_cutmix,flip,crop,affine")  # 데이터 증강 기법 추가 명시

# WandB 설정
config = {
    "model": "resnet50",
    "batch_size": 128,
    "num_epochs": 100,
    "learning_rate": 0.001,
    "optimizer": "Adam",
    "seed": 2025,
    "deterministic": False,
    "patience": 10,  # early stopping patience
    "train_ratio": 0.8,
    "val_ratio": 0.1,
    "test_ratio": 0.1,
    "cutmix_alpha": 1.0,  # CutMix 알파 파라미터 추가
    "cutmix_prob": 0.5,   # CutMix 적용 확률 추가
    "crop_padding": 4,    # RandomCrop 패딩 크기
    "crop_size": 32,      # RandomCrop 크기 (CIFAR-100 이미지 크기는 32x32)
    "affine_degrees": 10, # RandomAffine 회전 각도 범위
    "affine_translate": (0.1, 0.1),  # RandomAffine 이동 범위 (가로, 세로)
    "affine_scale": (0.9, 1.1),      # RandomAffine 확대/축소 범위
    "affine_shear": 10                # RandomAffine 전단 범위
}
wandb.config.update(config)

# CIFAR-100 데이터셋 로드
# RandomCrop 변환 추가
transform_train = transforms.Compose([
    transforms.RandomCrop(config["crop_size"], padding=config["crop_padding"]),  # 패딩 후 랜덤 크롭
    transforms.RandomHorizontalFlip(),  # 수평 뒤집기
    transforms.RandomAffine(
        degrees=config["affine_degrees"],           # 회전 각도 범위 (-15도 ~ 15도)
        translate=config["affine_translate"],       # 이동 범위 (가로, 세로)
        scale=config["affine_scale"],               # 확대/축소 범위
        shear=config["affine_shear"]                # 전단 범위
    ),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

full_trainset = torchvision.datasets.CIFAR100(
    root='./data', train=True, download=True, transform=transform_train)

testset = torchvision.datasets.CIFAR100(
    root='./data', train=False, download=True, transform=transform_test)

# Stratified 분할을 위한 준비 (train, validation 나누기)
# 모든 라벨을 추출
targets = np.array(full_trainset.targets)

# StratifiedShuffleSplit을 사용하여 8:2 비율로 분할
# train과 validation을 나눔 (full_trainset에서 8:2)
train_val_split = StratifiedShuffleSplit(n_splits=1, test_size=config["val_ratio"], random_state=config["seed"])
train_idx, val_idx = next(train_val_split.split(np.zeros(len(targets)), targets))

# Subset 생성
trainset = Subset(full_trainset, train_idx)
valset = Subset(full_trainset, val_idx)

# DataLoader 생성
trainloader = DataLoader(trainset, batch_size=config["batch_size"], shuffle=True, num_workers=16)
valloader = DataLoader(valset, batch_size=config["batch_size"], shuffle=False, num_workers=16)
testloader = DataLoader(testset, batch_size=config["batch_size"], shuffle=False, num_workers=16)

print(f"Train set size: {len(trainset)}")
print(f"Validation set size: {len(valset)}")
print(f"Test set size: {len(testset)}")

# 추가: CutMix 변환 정의
cutmix = transforms_v2.CutMix(alpha=config["cutmix_alpha"], num_classes=100)  # CIFAR-100은 100개 클래스

# CutMix용 손실 함수 정의 (원-핫 인코딩된 레이블 처리)
def cutmix_criterion(outputs, targets):
    """
    CutMix로 혼합된 레이블을 처리하기 위한 손실 함수
    outputs: 모델 출력
    targets: CutMix로 생성된 원-핫 인코딩 레이블
    """
    return torch.nn.functional.cross_entropy(outputs, targets)

def train(model, trainloader, criterion, optimizer, device, epoch):
    """
    학습 함수 (CutMix 적용)
    """
    model.train()   # 모델을 학습 모드로 설정
    start_time = time.time()  # 시간 측정 시작
    running_loss = 0.0
    correct_top1 = 0
    correct_top5 = 0
    total = 0
    
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # CutMix 확률적 적용
        if random.random() < config["cutmix_prob"]:
            inputs, labels = cutmix(inputs, labels)
            # 이 경우 labels은 원-핫 인코딩 형태로 변환됨
            use_cutmix = True
        else:
            use_cutmix = False
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        
        # CutMix 적용 여부에 따라 손실 함수 선택
        if use_cutmix:
            # CutMix가 적용된 경우 (원-핫 인코딩된 레이블)
            loss = torch.nn.functional.cross_entropy(outputs, labels)
        else:
            # 일반적인 경우 (정수 인덱스 레이블)
            loss = criterion(outputs, labels)
            
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # 정확도 계산 - CutMix 적용 여부에 따라 다르게 처리
        if use_cutmix:
            # 원-핫 인코딩된 레이블에서 argmax를 사용해 가장 큰 값의 인덱스 추출
            _, label_idx = labels.max(1)
        else:
            # 정수 인덱스 레이블 그대로 사용
            label_idx = labels
            
        # top-1 정확도 계산
        _, predicted = outputs.max(1)
        total += inputs.size(0)
        correct_top1 += predicted.eq(label_idx).sum().item()
        
        # top-5 정확도 계산
        _, top5_idx = outputs.topk(5, 1, largest=True, sorted=True)
        correct_top5 += sum([1 for i in range(len(label_idx)) if label_idx[i] in top5_idx[i]])
        
        if (i + 1) % 50 == 0:  # 50 배치마다 출력
            print(f'Epoch [{epoch+1}], Batch [{i+1}/{len(trainloader)}], Loss: {loss.item():.4f}')
    
    epoch_loss = running_loss / len(trainloader)
    accuracy_top1 = 100.0 * correct_top1 / total
    accuracy_top5 = 100.0 * correct_top5 / total
    
    train_time = time.time() - start_time
    
    # 학습 세트에 대한 성능 출력
    print(f'Train set: Epoch: {epoch+1}, Average loss:{epoch_loss:.4f}, LR: {optimizer.param_groups[0]["lr"]:.6f} '
          f'Top-1 Accuracy: {accuracy_top1:.4f}%, Top-5 Accuracy: {accuracy_top5:.4f}%, Time consumed:{train_time:.2f}s')
    
    return epoch_loss, accuracy_top1, accuracy_top5

def evaluate(model, dataloader, criterion, device, epoch, phase="val"):
    """
    평가 함수
    """
    model.eval()  # 모델을 평가 모드로 설정
    start_time = time.time()  # 시간 측정 시작
    
    eval_loss = 0.0
    correct_top1 = 0
    correct_top5 = 0
    total = 0
    
    # 그래디언트 계산 비활성화
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # 순전파
            outputs = model(inputs)
            
            # 손실 계산
            loss = criterion(outputs, labels)
            eval_loss += loss.item()
            
            # top-1 정확도 계산
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct_top1 += (predicted == labels).sum().item()
            
            # top-5 정확도 계산
            _, top5_idx = outputs.topk(5, 1, largest=True, sorted=True)
            correct_top5 += top5_idx.eq(labels.view(-1, 1).expand_as(top5_idx)).sum().item()
    
    # 평균 손실 및 정확도 계산
    eval_loss = eval_loss / len(dataloader)
    accuracy_top1 = 100.0 * correct_top1 / total
    accuracy_top5 = 100.0 * correct_top5 / total
    
    # 평가 시간 계산
    eval_time = time.time() - start_time
    
    # 테스트 세트에 대한 성능 출력
    print(f'{phase.capitalize()} set: Epoch: {epoch+1}, Average loss:{eval_loss:.4f}, '
          f'Top-1 Accuracy: {accuracy_top1:.4f}%, Top-5 Accuracy: {accuracy_top5:.4f}%, Time consumed:{eval_time:.2f}s')
    print()
    
    return eval_loss, accuracy_top1, accuracy_top5


# 메인 학습 루프
def main_training_loop(model, trainloader, valloader, testloader, criterion, optimizer, device, num_epochs, patience):
    """
    메인 학습 루프
    """
    early_stopping = EarlyStopping(patience=patience, verbose=True)
    
    best_acc_top1 = 0.0
    best_acc_top5 = 0.0
    
    # tqdm을 사용한 진행 상황 표시
    for epoch in tqdm(range(num_epochs)):
        # 학습
        train_loss, train_acc_top1, train_acc_top5 = train(model, trainloader, criterion, optimizer, device, epoch)
        
        # 평가
        val_loss, val_acc_top1, val_acc_top5 = evaluate(model, valloader, criterion, device, epoch, phase="val")
        
        # WandB에 로깅
        wandb.log({
            "epoch": epoch + 1,
            "learning_rate": optimizer.param_groups[0]['lr'],
            "train_loss": train_loss,
            "train_accuracy_top1": train_acc_top1,
            "train_accuracy_top5": train_acc_top5,
            "val_loss": val_loss,
            "val_accuracy_top1": val_acc_top1,
            "val_accuracy_top5": val_acc_top5
        })
            
        # 최고 정확도 모델 저장 (top-1 기준)
        if val_acc_top1 > best_acc_top1:
            best_acc_top1 = val_acc_top1
            best_acc_top5_at_best_top1 = val_acc_top5
            print(f'New best top-1 accuracy: {best_acc_top1:.2f}%, top-5 accuracy: {best_acc_top5_at_best_top1:.2f}%')
            # 모델 저장
            model_path = f'best_model_{wandb.run.name}.pth'
            torch.save(model.state_dict(), model_path)
            
            # WandB에 모델 아티팩트 저장
            wandb.save(model_path)
        
        # top-5 accuracy 기록 업데이트
        if val_acc_top5 > best_acc_top5:
            best_acc_top5 = val_acc_top5
            print(f'New best top-5 accuracy: {best_acc_top5:.2f}%')

        # Early stopping 체크 (validation loss 기준)
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping triggered. Training stopped.")
            break
    
    # 훈련 완료 후 모델 평가 (best model 로드)
    if early_stopping.early_stop:
        print("Loading best model from early stopping checkpoint...")
    else:
        print("Loading best model based on validation accuracy...")
        model_path = f'best_model_{wandb.run.name}.pth'
        model.load_state_dict(torch.load(model_path))

    # 최종 테스트 세트 평가
    test_loss, test_acc_top1, test_acc_top5 = evaluate(model, testloader, criterion, device, num_epochs-1, phase="test")
    
    # 테스트 결과를 wandb 로그에 추가
    wandb.log({
        "epoch": epoch + 1,  # 마지막 에폭 또는 early stopping된 에폭
        "test_loss": test_loss,
        "test_accuracy_top1": test_acc_top1,
        "test_accuracy_top5": test_acc_top5
    })
    
    print(f'Finish! Best validation top-1 accuracy: {best_acc_top1:.2f}%, Best validation top-5 accuracy: {best_acc_top5:.2f}%')
    print(f'Final test top-1 accuracy: {test_acc_top1:.2f}%, Final test top-5 accuracy: {test_acc_top5:.2f}%')
    
    # WandB에 최종 결과 기록
    wandb.run.summary["best_val_accuracy_top1"] = best_acc_top1
    wandb.run.summary["best_val_accuracy_top5"] = best_acc_top5
    wandb.run.summary["test_accuracy_top1"] = test_acc_top1
    wandb.run.summary["test_accuracy_top5"] = test_acc_top5

    # Early stopping 정보 저장
    if early_stopping.early_stop:
        wandb.run.summary["early_stopped"] = True
        wandb.run.summary["early_stopped_epoch"] = epoch+1
    else:
        wandb.run.summary["early_stopped"] = False


# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 모델 초기화
model = resnet50().to(device)  
criterion = nn.CrossEntropyLoss()  # 손실 함수 정의
optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])  # 옵티마이저 정의

# WandB에 모델 구조 기록
wandb.watch(model, log="all")

# GPU 가속
if torch.cuda.device_count() > 1:
    print(f"{torch.cuda.device_count()}개의 GPU를 사용합니다.")
    model = nn.DataParallel(model)

# 훈련 시작 시간 기록
start_time = time.time()

# 메인 학습 루프 호출
main_training_loop(
    model=model,
    trainloader=trainloader,
    valloader=valloader,
    testloader=testloader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    num_epochs=config["num_epochs"],
    patience=config["patience"]
)

# 훈련 종료 시간 기록 및 출력
end_time = time.time()
total_time = end_time - start_time
wandb.log({"total_training_time": total_time})

print(f"Total training time: {total_time:.2f} seconds")

# WandB 실행 종료
wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/guswls/.netrc
wandb: Currently logged in as: sokjh1310 (sokjh1310-hanyang-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Files already downloaded and verified
Files already downloaded and verified
Train set size: 45000
Validation set size: 5000
Test set size: 10000
Using device: cuda
2개의 GPU를 사용합니다.


  0%|                                                                                                       | 0/100 [00:00<?, ?it/s]

Epoch [1], Batch [50/352], Loss: 4.4192
Epoch [1], Batch [100/352], Loss: 4.2089
Epoch [1], Batch [150/352], Loss: 4.2307


  0%|                                                                                                       | 0/100 [00:40<?, ?it/s]


KeyboardInterrupt: 